# Timing tests

This notebook contains timing tests for `starry`, comparing it to the previous version of the code and other popular light curve modeling software.

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
# Install the beta version if needed
try:
    import starry_beta
except:
    !pip install starry_beta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import starry_beta
from tqdm import tqdm_notebook as tqdm
import time
from exoplanet import LimbDarkLightCurve
from exoplanet.orbits import KeplerianOrbit
import theano

starry.config.lazy = False
starry.config.quiet = True

## Compare to `beta`: $Y_{lm}$ phase curves and occultations

For light curves with fewer than a couple hundred points, the new version of ``starry`` is *slower* than the ``beta`` version. That's because the switch to ``theano`` adds a certain amount of overhead to every function call. This is mostly fine, since modern datasets tend to have far more data points; plus, you'll probably want to oversample the light curve anyways to account for the finite exposure time. That will get us firmly into the territory where the new version is comparable to or faster than the ``beta`` version.

In [ ]:
def time_flux(ydeg, occultation=False, npts=np.logspace(0, 4, 10), ntimes=30):

    # Define the new starry function
    map = starry.Map(ydeg=ydeg)
    map[1:, :] = 1
    map.inc = 45
    t_flux = lambda theta, xo, yo, ro: map.flux(theta=theta, xo=xo, yo=yo, ro=ro)

    # Define the starry beta function
    map_beta = starry_beta.Map(ydeg)
    map_beta[1:, :] = 1
    map_beta.axis = [0, 1, 1]
    b_flux = lambda theta, xo, yo, ro: map_beta.flux(theta=theta, xo=xo, yo=yo, ro=ro)

    if occultation:
        ro = 0.1
    else:
        ro = 0.0

    t_time = np.zeros_like(npts)
    b_time = np.zeros_like(npts)
    for i in range(len(npts)):

        theta = np.linspace(-180, 180, int(npts[i]))
        xo = np.linspace(-1.0, 1.0, int(npts[i]))
        yo = np.zeros_like(xo) + 0.1

        for t, flux in zip([t_time, b_time], [t_flux, b_flux]):
            elapsed = np.zeros(ntimes)
            for k in range(ntimes):
                tstart = time.time()
                flux(theta, xo, yo, ro)
                elapsed[k] = time.time() - tstart
            t[i] = np.median(elapsed[1:])

    return b_time, t_time

In [ ]:
ydeg = [1, 3, 5, 10]
npts = np.logspace(0, 4, 10)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 7))
ax = ax.flatten()

for i in range(len(ydeg)):
    for occultation, fillstyle, ls in zip([False, True], ["none", "full"], ["--", "-"]):
        b_time, t_time = time_flux(ydeg[i], npts=npts, occultation=occultation)
        ax[i].plot(npts, t_time, "C0o", fillstyle=fillstyle, ls="none", ms=3)
        ax[i].plot(npts, t_time, "C0", ls=ls, lw=1, alpha=0.5)
        ax[i].plot(npts, b_time, "C1o", fillstyle=fillstyle, ls="none", ms=3)
        ax[i].plot(npts, b_time, "C1", ls=ls, lw=1, alpha=0.5)
        ax[i].set_xscale("log")
        ax[i].set_yscale("log")
        ax[i].annotate(
            "l = %s" % ydeg[i],
            xy=(0, 1),
            xycoords="axes fraction",
            xytext=(5, -5),
            textcoords="offset points",
            ha="left",
            va="top",
            fontsize=12,
        )

ax[0].plot([], [], "C0-", label="this version")
ax[0].plot([], [], "C1-", label="beta")
ax[0].plot([], [], "k--", lw=1, label="rotation")
ax[0].plot([], [], "k-", lw=1, label="occultation")
ax[0].legend(fontsize=8, loc="upper right")
for i in [2, 3]:
    ax[i].set_xlabel("Number of points", fontsize=14)
for i in [0, 2]:
    ax[i].set_ylabel("Time [seconds]", fontsize=14)

## Compare to `beta`: Limb-darkened occultations

Since light curves of purely limb-darkened maps are so much faster to compute than those of spherical harmonic maps, the overhead is more apparent in the limb-darkened case. It's still a small price to pay for the integration with ``pymc3`` that the new version affords!

In [ ]:
def time_flux(udeg, npts=np.logspace(0, 4, 10), ntimes=30):

    # Define the new starry function
    map = starry.Map(udeg=udeg)
    map[1:] = 1
    t_flux = lambda xo, yo, ro: map.flux(xo=xo, yo=yo, ro=ro)

    # Define the starry beta function
    map_beta = starry_beta.Map(udeg)
    map_beta[1:] = 1
    b_flux = lambda xo, yo, ro: map_beta.flux(xo=xo, yo=yo, ro=ro)

    ro = 0.1
    t_time = np.zeros_like(npts)
    b_time = np.zeros_like(npts)
    for i in range(len(npts)):

        xo = np.linspace(-1.0, 1.0, int(npts[i]))
        yo = np.zeros_like(xo) + 0.1

        for t, flux in zip([t_time, b_time], [t_flux, b_flux]):
            elapsed = np.zeros(ntimes)
            for k in range(ntimes):
                tstart = time.time()
                flux(xo, yo, ro)
                elapsed[k] = time.time() - tstart
            t[i] = np.median(elapsed[1:])

    return b_time, t_time

In [ ]:
udeg = [1, 2, 3, 5]
npts = np.logspace(0, 4, 10)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 7))
ax = ax.flatten()

for i in range(len(udeg)):
    b_time, t_time = time_flux(udeg[i], npts=npts)
    ax[i].plot(npts, t_time, "C0o", ls="none", ms=3)
    ax[i].plot(npts, t_time, "C0", lw=1, alpha=0.5)
    ax[i].plot(npts, b_time, "C1o", ls="none", ms=3)
    ax[i].plot(npts, b_time, "C1", lw=1, alpha=0.5)
    ax[i].set_xscale("log")
    ax[i].set_yscale("log")
    ax[i].annotate(
        "l = %s" % udeg[i],
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(5, -5),
        textcoords="offset points",
        ha="left",
        va="top",
        fontsize=12,
    )

ax[0].plot([], [], "C0-", label="this version")
ax[0].plot([], [], "C1-", label="beta")
ax[0].legend(fontsize=8, loc="upper right")
for i in [2, 3]:
    ax[i].set_xlabel("Number of points", fontsize=14)
for i in [0, 2]:
    ax[i].set_ylabel("Time [seconds]", fontsize=14)

## Compare to `exoplanet`: Limb-darkened occultations

The algorithm used by ``exoplanet`` for computing occultations of limb-darkened stars is identical to the one used in ``starry``. But since ``starry`` is a lot more flexible than the light curve model in ``exoplanet``, it tends to be about a factor of 2 slower. This isn't a big deal in the grand scheme of things.

In [ ]:
def time_flux(udeg, npts=np.logspace(0, 4, 10), ntimes=30):

    # Define the starry function
    map = starry.Map(udeg=udeg)

    def t_flux(u, xo, yo, ro):
        map[1:] = u
        return map.flux(xo=xo, yo=yo, ro=ro)

    # Define the exoplanet function
    orbit = KeplerianOrbit(period=3.456)
    u = theano.tensor.dvector()
    t = theano.tensor.dvector()
    ro = theano.tensor.dscalar()
    b_flux = theano.function(
        [u, t, ro], 1.0 + LimbDarkLightCurve(u).get_light_curve(orbit=orbit, r=ro, t=t)
    )

    # Compute
    u = np.ones(udeg)
    ro = 0.1
    t_time = np.zeros_like(npts)
    b_time = np.zeros_like(npts)
    for i in range(len(npts)):

        t = np.linspace(-0.1, 0.1, int(npts[i]))
        xo, yo, _ = orbit.get_planet_position(t)
        xo = xo.eval()
        yo = yo.eval()

        # starry
        elapsed = np.zeros(ntimes)
        for k in range(ntimes):
            tstart = time.time()
            t_flux(u, xo, yo, ro)
            elapsed[k] = time.time() - tstart
        t_time[i] = np.median(elapsed)

        # exoplanet
        elapsed = np.zeros(ntimes)
        for k in range(ntimes):
            tstart = time.time()
            b_flux(u, t, ro)
            elapsed[k] = time.time() - tstart
        b_time[i] = np.median(elapsed[1:])

    return b_time, t_time

In [ ]:
udeg = [1, 2, 3, 5]
npts = np.logspace(0, 4, 10)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 7))
ax = ax.flatten()

for i in range(len(udeg)):
    b_time, t_time = time_flux(udeg[i], npts=npts)
    ax[i].plot(npts, t_time, "C0o", ls="none", ms=3)
    ax[i].plot(npts, t_time, "C0", lw=1, alpha=0.5)
    ax[i].plot(npts, b_time, "C1o", ls="none", ms=3)
    ax[i].plot(npts, b_time, "C1", lw=1, alpha=0.5)
    ax[i].set_xscale("log")
    ax[i].set_yscale("log")
    ax[i].annotate(
        "l = %s" % udeg[i],
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(5, -5),
        textcoords="offset points",
        ha="left",
        va="top",
        fontsize=12,
    )

ax[0].plot([], [], "C0-", label="starry")
ax[0].plot([], [], "C1-", label="exoplanet")
ax[0].legend(fontsize=8, loc="upper right")
for i in [2, 3]:
    ax[i].set_xlabel("Number of points", fontsize=14)
for i in [0, 2]:
    ax[i].set_ylabel("Time [seconds]", fontsize=14)